<a href="https://colab.research.google.com/github/nicoschmidt/DigitalCentones/blob/master/Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing Cento Christus Patiens using alignment algorithms

- get code and files from GitHub
- import code

In [15]:
!rm -r DigitalCentones
!git clone https://github.com/nicoschmidt/DigitalCentones

from DigitalCentones.smith_waterman import SmithWaterman
from DigitalCentones.alignment import *

Cloning into 'DigitalCentones'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 38 (delta 17), reused 18 (delta 6), pack-reused 0
Unpacking objects: 100% (38/38), done.


# Load Chritus Patiens and Medea text files
- copy xml file of Christus Patiens from GitHub repository ***First1KGreek*** by the ***OpenGreekAndLatin*** project
- parse the xml file
- remove preface lines
- load first 48 lines from Medea as plain text (no xml available yet :( )

In [11]:
# get Christus Patiens xml file
!wget https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml
lines_chr_pat = load_xml('tlg2022.tlg003.opp-grc1.xml')
for i in range(30, len(lines_chr_pat)):
    lines_chr_pat[i].no = -29+i # hack to re-number lines
lines_chr_pat = lines_chr_pat[30:]
print('Christus Patiens:', lines_chr_pat[:3],'\n\n')

lines_medea = load_plain_text('DigitalCentones/data/Medea-48.txt', 'Medea', 'Medea', 'Euripides')
print('Medea:', lines_medea[:3], '\n\n')

--2019-05-19 10:28:54--  https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/data/tlg2022/tlg003/tlg2022.tlg003.opp-grc1.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403903 (394K) [text/plain]
Saving to: ‘tlg2022.tlg003.opp-grc1.xml.6’

tlg2022.tlg003.opp- 100%[===================>] 394.44K  --.-KB/s    in 0.06s   

2019-05-19 10:28:54 (6.20 MB/s) - ‘tlg2022.tlg003.opp-grc1.xml.6’ saved [403903/403903]

Christus Patiens: [work_id:            urn:cts:greekLit:tlg2022.tlg003.opp-grc1
idx:                30
no:                 1
speaker:            ΘΕΟΤΟΚΟΣ.
text_raw:           Εἴθ᾿ ὤφελ’ ἐν λειμῶνι μηδ’ ἕρπειν ὄφις,
text:               εἴθ ὤφελ ἐν λειμῶνι μηδ ἕρπειν ὄφις
text_no_diacritics: ειθ ωφελ εν λειμωνι μηδ ερπειν οφις
text_lemma: 




---

**Print all characters with diacritics**



In [13]:
# print all characters in the corpus:
alphabet = np.unique([c for lines in [lines_chr_pat, lines_medea] for line in lines for c in line.text])
print('ID, Character, Name')
for a in alphabet:
    print('{} "{}" {}'.format(hex(ord(a)), a, unicodedata.name(a, 'not defined')))

ID, Character, Name
0x20 " " SPACE
0xe1 "á" LATIN SMALL LETTER A WITH ACUTE
0x390 "ΐ" GREEK SMALL LETTER IOTA WITH DIALYTIKA AND TONOS
0x3ac "ά" GREEK SMALL LETTER ALPHA WITH TONOS
0x3ad "έ" GREEK SMALL LETTER EPSILON WITH TONOS
0x3ae "ή" GREEK SMALL LETTER ETA WITH TONOS
0x3af "ί" GREEK SMALL LETTER IOTA WITH TONOS
0x3b1 "α" GREEK SMALL LETTER ALPHA
0x3b2 "β" GREEK SMALL LETTER BETA
0x3b3 "γ" GREEK SMALL LETTER GAMMA
0x3b4 "δ" GREEK SMALL LETTER DELTA
0x3b5 "ε" GREEK SMALL LETTER EPSILON
0x3b6 "ζ" GREEK SMALL LETTER ZETA
0x3b7 "η" GREEK SMALL LETTER ETA
0x3b8 "θ" GREEK SMALL LETTER THETA
0x3b9 "ι" GREEK SMALL LETTER IOTA
0x3ba "κ" GREEK SMALL LETTER KAPPA
0x3bb "λ" GREEK SMALL LETTER LAMDA
0x3bc "μ" GREEK SMALL LETTER MU
0x3bd "ν" GREEK SMALL LETTER NU
0x3be "ξ" GREEK SMALL LETTER XI
0x3bf "ο" GREEK SMALL LETTER OMICRON
0x3c0 "π" GREEK SMALL LETTER PI
0x3c1 "ρ" GREEK SMALL LETTER RHO
0x3c2 "ς" GREEK SMALL LETTER FINAL SIGMA
0x3c3 "σ" GREEK SMALL LETTER SIGMA
0x3c4 "τ" GREEK SMALL LETT

**Print all characters without diacritics**

In [14]:
alphabet_no_diacritics = np.unique([c for lines in [lines_chr_pat, lines_medea] for line in lines for c in line.text_no_diacritics])
for a in alphabet_no_diacritics:
    print('{} "{}" {}'.format(hex(ord(a)), a, unicodedata.name(a, 'not defined')))


0x20 " " SPACE
0x61 "a" LATIN SMALL LETTER A
0x3b1 "α" GREEK SMALL LETTER ALPHA
0x3b2 "β" GREEK SMALL LETTER BETA
0x3b3 "γ" GREEK SMALL LETTER GAMMA
0x3b4 "δ" GREEK SMALL LETTER DELTA
0x3b5 "ε" GREEK SMALL LETTER EPSILON
0x3b6 "ζ" GREEK SMALL LETTER ZETA
0x3b7 "η" GREEK SMALL LETTER ETA
0x3b8 "θ" GREEK SMALL LETTER THETA
0x3b9 "ι" GREEK SMALL LETTER IOTA
0x3ba "κ" GREEK SMALL LETTER KAPPA
0x3bb "λ" GREEK SMALL LETTER LAMDA
0x3bc "μ" GREEK SMALL LETTER MU
0x3bd "ν" GREEK SMALL LETTER NU
0x3be "ξ" GREEK SMALL LETTER XI
0x3bf "ο" GREEK SMALL LETTER OMICRON
0x3c0 "π" GREEK SMALL LETTER PI
0x3c1 "ρ" GREEK SMALL LETTER RHO
0x3c2 "ς" GREEK SMALL LETTER FINAL SIGMA
0x3c3 "σ" GREEK SMALL LETTER SIGMA
0x3c4 "τ" GREEK SMALL LETTER TAU
0x3c5 "υ" GREEK SMALL LETTER UPSILON
0x3c6 "φ" GREEK SMALL LETTER PHI
0x3c7 "χ" GREEK SMALL LETTER CHI
0x3c8 "ψ" GREEK SMALL LETTER PSI
0x3c9 "ω" GREEK SMALL LETTER OMEGA


# Load all Matches found by Toullier

In [18]:
# Input file is a text file with one header row and remaining rows structured as
# '<target_line_number>,<source_work_number>,<source_work_line_number>...\n'
line_to_line_map = np.loadtxt('DigitalCentones/data/ChristusPatiens_Tulliers_matches.csv', int, delimiter=',', skiprows=1, usecols=[0,1,2])

# These are all matched works Toullier found
source_work_names = ['Agamemnon',
                     'Prometheus',
                     'Alkestis',
                     'Andromache',
                     'Bakchen',
                     'Hekabe',
                     'Helena',
                     'Hippolytos',
                     'Iphigenie in Aulis',
                     'Iphigenie in Tauris',
                     'Medea',
                     'Orestes',
                     'Ph\"onikerinnen',
                     'Rhesos',
                     'Troerinnen',
                     'Ilias',
                     'Alexandra']

# store map as a dictionary with target_line_number as keys and tuples (<source_work_id>, <source_work_number>) as values 
line_to_line_map = dict([(target_line_no,(source_work_names[source_work_id-1], source_line_id)) for target_line_no, source_work_id, source_line_id in line_to_line_map])
line_to_line_map

{1: ('Medea', 1),
 2: ('Medea', 3),
 3: ('Medea', 6),
 4: ('Medea', 20),
 5: ('Hekabe', 1123),
 6: ('Medea', 8),
 8: ('Medea', 9),
 14: ('Medea', 10),
 15: ('Medea', 11),
 17: ('Prometheus', 1027),
 25: ('Alexandra', 151),
 32: ('Medea', 14),
 33: ('Medea', 15),
 34: ('Medea', 13),
 37: ('Medea', 16),
 38: ('Medea', 17),
 39: ('Agamemnon', 764),
 40: ('Troerinnen', 605),
 41: ('Troerinnen', 620),
 42: ('Troerinnen', 621),
 43: ('Medea', 20),
 45: ('Prometheus', 1027),
 46: ('Medea', 25),
 47: ('Medea', 26),
 50: ('Hippolytos', 450),
 51: ('Medea', 21),
 52: ('Medea', 22),
 53: ('Medea', 34),
 54: ('Medea', 35),
 55: ('Medea', 36),
 56: ('Medea', 56),
 57: ('Medea', 57),
 58: ('Medea', 58),
 59: ('Medea', 59),
 61: ('Hekabe', 736),
 65: ('Agamemnon', 611),
 66: ('Agamemnon', 612),
 71: ('Agamemnon', 587),
 72: ('Agamemnon', 588),
 73: ('Agamemnon', 589),
 75: ('Agamemnon', 593),
 76: ('Agamemnon', 591),
 77: ('Troerinnen', 747),
 78: ('Troerinnen', 748),
 79: ('Agamemnon', 594),
 80: ('

{1: ('Medea', 1),
 2: ('Medea', 3),
 3: ('Medea', 6),
 4: ('Medea', 20),
 5: ('Hekabe', 1123),
 6: ('Medea', 8),
 8: ('Medea', 9),
 14: ('Medea', 10),
 15: ('Medea', 11),
 17: ('Prometheus', 1027),
 25: ('Alexandra', 151),
 32: ('Medea', 14),
 33: ('Medea', 15),
 34: ('Medea', 13),
 37: ('Medea', 16),
 38: ('Medea', 17),
 39: ('Agamemnon', 764),
 40: ('Troerinnen', 605),
 41: ('Troerinnen', 620),
 42: ('Troerinnen', 621),
 43: ('Medea', 20),
 45: ('Prometheus', 1027),
 46: ('Medea', 25),
 47: ('Medea', 26),
 50: ('Hippolytos', 450),
 51: ('Medea', 21),
 52: ('Medea', 22),
 53: ('Medea', 34),
 54: ('Medea', 35),
 55: ('Medea', 36),
 56: ('Medea', 56),
 57: ('Medea', 57),
 58: ('Medea', 58),
 59: ('Medea', 59),
 61: ('Hekabe', 736),
 65: ('Agamemnon', 611),
 66: ('Agamemnon', 612),
 71: ('Agamemnon', 587),
 72: ('Agamemnon', 588),
 73: ('Agamemnon', 589),
 75: ('Agamemnon', 593),
 76: ('Agamemnon', 591),
 77: ('Troerinnen', 747),
 78: ('Troerinnen', 748),
 79: ('Agamemnon', 594),
 80: ('